In [2]:
from __future__ import print_function

import numpy as np
import math as math
import time
from time import time_ns
from decimal import *

def benchmark(iterations, difficulty):
    
    np.random.seed(0)# Let's take the randomness out of random numbers (for reproducibility)
    map = {}
    
    #==========================================================================#
    # Matrix multiplication
    N = 1 * iterations
    size = 1 * difficulty
    #-------------
    A, B = np.random.random((size, size)).astype('float64'), np.random.random((size, size)).astype('float64')
    t = Decimal(time_ns())
    for i in range(N): np.dot(A, B)
    delta = (Decimal(time_ns()) - t)/ (10 ** 9)
    map["matrix_multiplication"] = [delta]
    del A, B
    #==========================================================================#
    # Vector multiplication
    N = 1 * iterations
    size = 1 * difficulty**2
    #-------------
    C, D = np.random.random((size,)).astype('float64'), np.random.random((size,)).astype('float64')
    t = Decimal(time_ns())
    for i in range(N): np.dot(C, D)
    delta = (Decimal(time_ns()) - t)/ (10 ** 9)
    map["vector_multiplication"] = [delta]
    del C, D
    #==========================================================================#
    # Manual Convolution
    N = 1 * iterations
    size = 1 * difficulty
    #-------------
    a = np.arange(size*size, dtype=np.float64).reshape(size,size)
    t = Decimal(time_ns())
    for i in range(N):
        rowconvol = a[1:-1,:] + a[:-2,:] + a[2:,:]
        colconvol = rowconvol[:,1:-1] + rowconvol[:,:-2] + rowconvol[:,2:] - 9*a[1:-1,1:-1]
    delta = (Decimal(time_ns()) - t)/ (10 ** 9)
    map["manual_convolution"] = [delta]
    del a
    #==========================================================================#
    # Tensor Math
    N = 1 * iterations
    size = 1 * difficulty
    #-------------
    dim = [math.floor(size/10), math.floor(size/6), math.floor(size/3)]
    if(dim[0] == 0): dim[0] = 2
    if(dim[1] == 0): dim[1] = 2
    if(dim[2] == 0): dim[2] = 1
    t1 = np.arange(np.prod(dim), dtype=np.float64).reshape(tuple(dim))
    t2 = np.arange(np.prod(dim), dtype=np.float64).reshape(tuple(dim))
    t = Decimal(time_ns())
    for i in range(N):
        v = t1 * 10
        v = v * t2 / t1
        v = v ** 0.5
    delta = (Decimal(time_ns()) - t)/ (10 ** 9)
    map["tensor_math"] = [delta]
    del t1, t2
    #==========================================================================#
    map["iterations"] = [iterations]
    map["difficulty"] = [difficulty] 
    #==========================================================================#
    #print('')
    #print('This was obtained using the following Numpy configuration:')
    #np.__config__.show()
    
    return map
    
def benchmark_session(conf, filename) :
    map = {}
    for i in range(conf["difficulty"],1+conf["difficulty"]+conf["sample_size"]):
        result = benchmark(
            conf["iterations"], 
            conf["difficulty"]+(i-conf["difficulty"])*conf["intensifier"]
        )
        for k, v in result.items() :
            if k in map :
                map[k] += v
            else :
                map[k] = v
    asCSV = ""
    ci = 0
    rowSize = len(map)
    for k, v in map.items() :
        asCSV += str(k)
        if ci < rowSize-1 : asCSV += ","
        ci = ci+1
    asCSV += "\n"
    for i in range(1,1+conf["sample_size"]):
        ci = 0
        for k, v in map.items() :
            asCSV += str(v[i-1])
            if ci < rowSize-1 : asCSV += ","
            ci = ci+1
        asCSV += "\n"
    print(asCSV)
    text_file = open(filename, "w")
    text_file.write(asCSV)
    text_file.close()
    
benchmark_session(
    {
       "iterations":1,
       "sample_size":20,
       "difficulty":15,
       "intensifier":50
    }, 
    "numpy_bench_CPU.csv"
)

    

<ipython-input-2-a7dfcfc3afdf>:63: RuntimeWarning: invalid value encountered in true_divide
  v = v * t2 / t1


matrix_multiplication,vector_multiplication,manual_convolution,tensor_math,iterations,difficulty
0,0,0,0,1,15
0,0,0,0,1,65
0.0009966,0,0,0,1,115
0,0.0007762,0,0,1,165
0.0009969,0.0006956,0,0.0009983,1,215
0.0009974,0.0012756,0.0019967,0.0009963,1,265
0.0009971,0,0.0020283,0.0029915,1,315
0.0019601,0.0012409,0.0039912,0.0039891,1,365
0.0009983,0,0.0039893,0.0079791,1,415
0.0019948,0.0010013,0.0050158,0.0089751,1,465
0.0029932,0.0007488,0.0060232,0.0109707,1,515
0.0029607,0.0009597,0.0059853,0.0149566,1,565
0.0059605,0.0009966,0.009956,0.0209525,1,615
0.0049549,0.0007662,0.0119677,0.0249331,1,665
0.0059858,0,0.0119681,0.031949,1,715
0.0069812,0.000999,0.0129652,0.0378985,1,765
0.0079811,0,0.0149601,0.045877,1,815
0.0099771,0.0009984,0.0159576,0.0538575,1,865
0.0109369,0,0.0189495,0.0638292,1,915
0.0129701,0.0009642,0.021939,0.0797853,1,965

